# Programming for Data Science and Artificial Intelligence

## 18. PyTorch

- [WEIDMAN] Ch7
- https://pytorch.org/tutorials/
- https://github.com/yunjey/pytorch-tutorial

Here we introduce PyTorch, an increasingly popular neural network framework based on **automatic differentiation**, which we introduced at the beginning of previous chapter.

As in the rest of the book, we’ll write our code in a way that maps to the mental models of how neural networks work, writing classes for Layers, Trainers, and so on. In doing so, we won’t be writing our code in line with common PyTorch practices, but I recommend you to watch the 60-min blitz to understand different features of pytorch (https://pytorch.org/tutorials/).  Pytorch is really cool and simple!  For example, there is a function <code>tensor.to(cuda)</code> so you can use the GPU to run any model which is much faster.  I highly recommend checking it out.

### Basics

1. Basic autograd example               
2. Loading data from numpy                
3. Input pipline                          

#### 1. Basic autograd example 1

In [1]:
import torch 

# Create tensors.
# only tensors of floating point dtype can get gradient
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients
# Pytorch tensor can automatically compute the derivative
# of the parameters in respect to loss
y.backward()

# Print out the gradients.
print("Gradient of x: ", x.grad)    # x.grad = 2 
print("Gradient of w: ", w.grad)    # w.grad = 1 
print("Gradient of b: ", b.grad)    # b.grad = 1 

Gradient of x:  tensor(2.)
Gradient of w:  tensor(1.)
Gradient of b:  tensor(1.)


#### 2. Loading data from numpy

In [2]:
import numpy as np

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

#### 3. Input pipeline

In [3]:
import torchvision
import torchvision.transforms as transforms

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='data',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

Files already downloaded and verified


In [4]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [5]:
# Retrieve batch of data
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

print(images.size())
print(labels.size())

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [6]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

### Linear Regression

Let's have linear regression as a case study to study the different components of pyTorch.  You will fall in love with it (at least, compared to writing from scratch!).  These are the following components we will be covering:

1. Specifying input and target
2. Dataset and DataLoader
3. nn.Linear (Dense)
4. Define loss function
5. Define optimizer function
6. Train the model

Consider this data:

<img src = "figures/japan.png">

n a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

$$yield_{apple}  = w_{11} * temp + w_{12} * rainfall + w_{13} * humidity + b_{1}$$

$$yield_{orange} = w_{21} * temp + w_{22} * rainfall + w_{23} * humidity + b_{2}$$

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

<img src = "figures/japan2.png">

The learning part of linear regression is to figure out a set of weights <code>w11, w12,... w23, b1 \& b2</code> by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called gradient descent

#### 1. Specifiying input and target

In [7]:
# Input (temp, rainfall, humidity)
x_train = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
y_train = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
print(inputs.size())
print(targets.size())

torch.Size([15, 3])
torch.Size([15, 2])


#### 2. Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [8]:
from torch.utils.data import TensorDataset

In [9]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation (<code>[0:3]</code> in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a <code>DataLoader</code>, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [10]:
from torch.utils.data import DataLoader

In [11]:
# Define data loader
batch_size = 15
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a for-in loop. Let's look at an example

In [12]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[119., 133.],
        [103., 119.],
        [103., 119.],
        [119., 133.],
        [ 81., 101.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 56.,  70.],
        [ 22.,  37.],
        [103., 119.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

#### 3. Define some layer - nn.Linear (same as Dense)

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [13]:
import torch.nn as nn

# Define model
model = nn.Linear(3, 2)  #nn.Linear assume this shape (in_features, out_features)
print(model.weight)
print(model.weight.size()) # (out_features, in_features)
print(model.bias)
print(model.bias.size()) #(out_features)

Parameter containing:
tensor([[ 0.2724,  0.2703, -0.3287],
        [-0.2303, -0.0214, -0.3860]], requires_grad=True)
torch.Size([2, 3])
Parameter containing:
tensor([-0.3404, -0.3438], requires_grad=True)
torch.Size([2])


In fact, our model is simply a function that performs a matrix multiplication of the <code>inputs</code> and the weights <code>w</code> and adds the bias <code>b</code> (for each observation)

<img src = "figures/dot.png" width="400">

PyTorch models also have a helpful <code>.parameters</code> method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [14]:
# Parameters
list(model.parameters())  #model.param returns a generator

[Parameter containing:
 tensor([[ 0.2724,  0.2703, -0.3287],
         [-0.2303, -0.0214, -0.3860]], requires_grad=True),
 Parameter containing:
 tensor([-0.3404, -0.3438], requires_grad=True)]

We can use the <code>model(tensor)</code> API to perform a forward-pass that generate predictions

In [15]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 23.5163, -35.1880],
        [ 27.1915, -47.8889],
        [ 40.5063, -45.6340],
        [ 26.9019, -39.0392],
        [ 21.3885, -45.3084],
        [ 23.5163, -35.1880],
        [ 27.1915, -47.8889],
        [ 40.5063, -45.6340],
        [ 26.9019, -39.0392],
        [ 21.3885, -45.3084],
        [ 23.5163, -35.1880],
        [ 27.1915, -47.8889],
        [ 40.5063, -45.6340],
        [ 26.9019, -39.0392],
        [ 21.3885, -45.3084]], grad_fn=<AddmmBackward>)

#### 4. Define loss function

The <code>nn</code> module contains a lot of useful loss function like this:

In [16]:
criterion_mse = nn.MSELoss()
criterion_softmax_cross_entropy_loss = nn.CrossEntropyLoss()

In [17]:
mse = criterion_mse(preds, targets)
print(mse)
print(mse.item())  ##print out the loss number

tensor(11471.8027, grad_fn=<MseLossBackward>)
11471.802734375


#### 5. Define the optimizer

*Learning rate and momentum*

Instead of manually manipulating the model's weights & biases using gradients, we can use <code>torch.optim</code> API.  We can use <code>optim.SGD</code> to perform stochastic gradient descent where samples are selected in batches (often with random shuffling) instead of as a single group.

Note that model.parameters() is passed as an argument to optim.SGD, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

In [18]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

Let's talk a bit about other techniques we have used before and how they look like in PyTorch.  Since this is simple linear regression problem, we only use learning rate and momentum, but it's worth to talk it here and perhaps we can see the code in the neural network section

*Weight initialization*

We don’t need to worry about weight initialization at all: the weights in most PyTorch operations involving parameters, including nn.Linear, are automatically scaled based on the size of the layer.

*Dropout*

Dropout is similarly easy. Just as PyTorch has a built-in Module <code>nn.Linear(n_in, n_out)</code> that computes the operations of a Dense layer from before, the Module nn.Dropout(dropout_prob) implements the Dropout operation, with the caveat that the probability passed in is by default the probability of dropping a given neuron.

*Learning rate decay*

PyTorch has an <code>lr_scheduler</code> class that can be used to decay the learning rate over the epochs. The key import you need to get started is <code>from torch.optim import lr_scheduler</code>   We shall use it in the latter section

#### 6. Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Forward pass
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient

In [19]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
                    
            # 1. Forward pass
            pred = model(xb)
                      
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Backward and optimize
            opt.zero_grad()  #if not, the gradients will accumlate
            loss.backward()
            
            # Print out the gradients.
            #print ('dL/dw: ', model.weight.grad) 
            #print ('dL/db: ', model.bias.grad)
            
            # 4. Update parameters using gradients
            opt.step()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

- We use the data loader defined earlier to get batches of data for every iteration.
- Instead of updating parameters (weights and biases) manually, we use <code>opt.step</code> to perform the update, and <code>opt.zero_grad</code> to reset the gradients to zero.
- We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [20]:
fit(100, model, criterion_mse, opt, train_dl)

Epoch [10/100], Loss: 431.4234
Epoch [20/100], Loss: 340.3797
Epoch [30/100], Loss: 611.1182
Epoch [40/100], Loss: 330.4228
Epoch [50/100], Loss: 88.4461
Epoch [60/100], Loss: 9.2291
Epoch [70/100], Loss: 0.6611
Epoch [80/100], Loss: 2.4046
Epoch [90/100], Loss: 2.0369
Epoch [100/100], Loss: 1.0624


In [21]:
# Generate predictions
preds = model(inputs)
loss = criterion_mse(preds, targets)
print(loss.item())

0.564578652381897


### Fully-Connected Neural Network

Let's load the MNIST dataset.  Our architecture is simple:

1. Input layer receiving 784 features
2. Hidden layer with size of 89 neurons
3. Output layer with size of 10 neurons

We will be using Sigmoid activation.

In [22]:
# Device configuration
# cuda refers to any NVIDIA GPU that you can use to run your code
# it will be much faster
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# Hyper-parameters 
input_size = 784
hidden_size = 89
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [30]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Let's define a fully-connected neural network with one hidden layer.  Actually, you can use nn.Sequential to easily do this.  I will be showing you how to do this using a class way.

In [26]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()  #super(Model, self)
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out

Let's now define the model using the class.  Every <code>nn.Module</code> can also use the <code>.to(device)</code> to fully use the GPU capabilities.

In [27]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

Let's define the Loss and optimizer.

Here we will be using Adam which is an adaptive learning rate optimization.  Comparing Adam and SGD, Adam is more adaptive in terms of how it uses momentum and learning rate.  Namely, Adam uses the **squared gradients to scale the learning rate** and it takes advantage of momentum by using **moving average of the gradient** instead of gradient itself like SGD with momentum

Whether Adam vs. SGD is still very debatable. Adam is proposed in 2015 to great success and many recent papers found that SGD can be more generalized than Adam...so I really don't know.  It's best to try both, I guess.

In [28]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()  #this is softmax indeed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

Let's train the model

In [35]:
# Train the model
total_step = len(train_loader)  #for printing purpose
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        
        #images shape is [100, 1, 28, 28]
        
        # Move tensors to the configured device
        # also reshape to [100, 784] so it can be inputted into the Dense layer
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
                
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0703
Epoch [1/5], Step [200/600], Loss: 0.1169
Epoch [1/5], Step [300/600], Loss: 0.0846
Epoch [1/5], Step [400/600], Loss: 0.0382
Epoch [1/5], Step [500/600], Loss: 0.0444
Epoch [1/5], Step [600/600], Loss: 0.0731
Epoch [2/5], Step [100/600], Loss: 0.0437
Epoch [2/5], Step [200/600], Loss: 0.0551
Epoch [2/5], Step [300/600], Loss: 0.0686
Epoch [2/5], Step [400/600], Loss: 0.0851
Epoch [2/5], Step [500/600], Loss: 0.0555
Epoch [2/5], Step [600/600], Loss: 0.0577
Epoch [3/5], Step [100/600], Loss: 0.0283
Epoch [3/5], Step [200/600], Loss: 0.0627
Epoch [3/5], Step [300/600], Loss: 0.1248
Epoch [3/5], Step [400/600], Loss: 0.0249
Epoch [3/5], Step [500/600], Loss: 0.0338
Epoch [3/5], Step [600/600], Loss: 0.0689
Epoch [4/5], Step [100/600], Loss: 0.0848
Epoch [4/5], Step [200/600], Loss: 0.0238
Epoch [4/5], Step [300/600], Loss: 0.0693
Epoch [4/5], Step [400/600], Loss: 0.0310
Epoch [4/5], Step [500/600], Loss: 0.0303
Epoch [4/5], Step [600/600], Loss:

Let's test the model

In [44]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  #returns max value, indices
        total += labels.size(0)  #keep track of total
        correct += (predicted == labels).sum().item()  #.item() give the raw number

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'models/dense-mnist.ckpt')

Accuracy of the network on the 10000 test images: 97.42 %


### Convolutional Neural Network

Here we will be exploring how to use pyTorch for CNN.

We will also be exploring more typical CNN architectures, as opposed to only simple one conv layer that we use earlier.

In [47]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Typical CNN architectures:

1. Definitely all CNN will put some Conv layer.  **Number of channels** are often a power of 2, because this often results in more efficient processing. The more channels you have, the greater granularity you give to the network.   Also, for **filter size**, typically it will be 3 or 5, where in general, smaller filter size gives better results, while larger filer size speed up the training.  For image input, it is desirable to have **squared images** because they are easy to work with the squared filter.  In the case with non-squared images, we can use the squared-filter to extract square patches anyway.  As for **stride**, it is most common to use stride sizes of 1 in most settings.  At most, we use 2 but no more higher.  **Padding** is selected using the formula $(filtersize - 1)/2$ to maintain same spatial footprint.

2. 

3. Convolution operation works well with ReLu activation.  ReLu has many advantages.  It is faster to compute, and its derivative is also fast to compute.  It has non-saturation of gradient (which mean the gradient does not vanish) since its derivative is 1

4. Back then, convolution operation typically use with max-pooling.  Aside from reduction of image size, it also help extract sharpest features of an image as for max pooling, and extract smooth features for average pooling.   Anyhow, as we said in earlier lesson, max pooling works somehow but there are also experiments showing worse results.  You may want to replace max-pooling with a convolution layer with stride of 2 and see whether it change anything   (28 - 2)/(2) + 1 = 26/2 = 13

In [84]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #in_channel = 1
            #out_channel = 16
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU())
            nn.MaxPool2d(kernel_size=2, stride=2))
        #after layer 1 will be of shape [100, 16, 14, 14]
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #after layer 2 will be of shape [100, 32, 7, 7]
        self.fc = nn.Linear(7*7*32, num_classes)
        #after fc will be of shape [100, 10]
        
    def forward(self, x):
        #x shape: [batch, in_channel, img_width, img_height]
        #[100, 1, 28, 28]
        out = self.layer1(x)
        print(out.size())
        #after layer 1: shape: [100, 16, 14, 14]
        #out = self.layer2(out)
        #after layer 2: shape: [100, 32, 7, 7]
        #out = out.reshape(out.size(0), -1)
        #after squeezing: shape: [100, 1568]
        #we squeeze so that it can be inputted into the fc layer
        #out = self.fc(out)
        #after fc layer: shape: [100, 10]
        return out


Define the model

In [85]:
model = ConvNet(num_classes).to(device)

Define the loss and optimizer

In [86]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [87]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #con2d expects (batch, channel, width, height)
        images = images.to(device)
        labels = labels.to(device)
        
        #print(images.size())
        #print(labels.size())
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

torch.Size([100, 16, 28, 28])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

Testing the code.  Same code.

<code>.eval()</code> ....

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'models/cnn.ckpt')